In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import cv2, os, tensorflow, keras
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image
from skimage import color
from google.colab.patches import cv2_imshow
import albumentations as A
from scipy.io import savemat
import scipy.io as sio
from numpy import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten, BatchNormalization, MaxPooling2D, Activation, Reshape
from sklearn.metrics import mean_squared_error
#image.ImageDataGenerator
import pickle

## **FOLDERS OF AUGMENTED IMAGES AND LABELS DATA**

In [ ]:
keyPointFiles='/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/ACombFile/KEYPOINT/'
trainImagesFiles='/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/ACombFile/IMG/'

## **IMAGES PREPROCESSING**

In [ ]:
def imageProcessing(imgFile):
  imgLoad=cv2.imread(imgFile)
  #converting rgb to gray
  imgGrey=cv2.cvtColor(imgLoad, cv2.COLOR_BGR2GRAY)
  #equalizing intensities
  imgHist=cv2.equalizeHist(imgGrey)
  #removing noise
  imgNoiseLess= cv2.fastNlMeansDenoising(imgHist, None, 10, 7, 21) 
  #Normalizing
  imgNorm=(imgNoiseLess - np.min(imgNoiseLess)) / (np.max(imgNoiseLess) - np.min(imgNoiseLess))
  return imgNorm

def KeyPointProcess(kpFile):
  test = sio.loadmat(kpFile)
  keyPoint=list(test.values())[3]
  return keyPoint

In [ ]:
imgList=os.listdir(trainImagesFiles)
print(f'TOTAL IMAGE FILES FOR TRAINING:{len(imgList)}')

TOTAL IMAGE FILES FOR TRAINING:2880


In [ ]:
#image Preprocessing and Histogram Equalization
imgList=os.listdir(trainImagesFiles)
imgProcess=[]
kptProcess=[]

for img in imgList:
  imgFile=trainImagesFiles+img
  kpFile=keyPointFiles+img+'.mat'
  #Extracting the keypoints
  keyPoint=np.asarray(KeyPointProcess(kpFile))
  kptProcess.append(keyPoint)
  #Extracting the images
  imgArray=imageProcessing(imgFile)
  imgProcess.append(imgArray)

#print(f'KEYPOINT SHAPE:{np.asarray(kptProcess).shape}')
#print(f'IMAGES SHAPE:{np.array(imgProcess).shape}')

## **PICKLING THE PREPROCESS IMAGES AND LABELLED FILES**

In [ ]:
'''featurefile=open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/IMG/Fimg2880_denoise_hist.pickle', 'wb')
pickle.dump(imgProcess,featurefile)
featurefile.close()
'''
IMGfile=pickle.load(open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/IMG/Fimg2880_denoise_hist.pickle','rb'))
imgA=np.asarray(IMGfile)
print(imgA.shape)

(2880, 128, 64)


In [ ]:
'''featurefile=open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/KEYPOINT/Fkp2800_denoise_hist.pickle', 'wb')
pickle.dump(kptProcess, featurefile)
featurefile.close()
'''
KPfile=pickle.load(open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/KEYPOINT/Fkp2800_denoise_hist.pickle','rb'))
kpA=np.asarray(KPfile)
print(kpA.shape)

(2880, 68, 2)


In [ ]:
plt.figure(figsize=(30,30))
i,idx=1,16
for num in range(0,16):
  #idx=np.random.choice(kpA.shape[0])
  plt.subplot(4,4,i)
  plt.imshow(imgA[idx])
  x,y=kpA[idx][:,0], kpA[idx][:,1]
  plt.scatter(x, y, color='red', s=10)
  plt.title(idx)
  i+=1
  idx+=1

In [ ]:
plt.figure(figsize=(30,30))
i,idx=1,1
for num in range(0,16):
  #idx=np.random.choice(kpA.shape[0])
  plt.subplot(4,4,i)
  plt.imshow(imgA[idx])
  x,y=kpA[idx][:,0], kpA[idx][:,1]
  plt.scatter(x, y, color='red', s=10)
  plt.title(idx)
  i+=1
  idx+=1

## **TEST IMAGES AND LABEL FOLDERS**

In [ ]:
TestImgFile='/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Test/IMG/'
TestKpFile='/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Test/KEYPOINT/'

## **TEST IMAGES PREPROCESSING**

In [ ]:
def imageProcessing(imgFile):
  imgLoad=cv2.imread(imgFile)
  #imgGrey=cv2.cvtColor(imgLoad, cv2.COLOR_BGR2GRAY)
  #imgHist=cv2.equalizeHist(imgGrey)
  #imgNoiseLess= cv2.fastNlMeansDenoising(imgHist, None, 10, 7, 21) 
  imgNorm=(imgNoiseLess - np.min(imgNoiseLess)) / (np.max(imgNoiseLess) - np.min(imgNoiseLess))
  #imgNorm=(imgHist-np.min(imgHist)) / (np.max(imgHist) - np.min(imgHist))
  return imgNorm

def KeyPointProcess(kpFile):
  test = sio.loadmat(kpFile)
  keyPoint=list(test.values())[3]
  return keyPoint


In [ ]:
#image Preprocessing and Histogram Equalization
imgList=os.listdir(TestImgFile)
imgProcess=[]
kptProcess=[]

for img in imgList:
  imgFile=TestImgFile+img
  kpFile=TestKpFile+img+'.mat'
  #Extracting the keypoints
  keyPoint=np.asarray(KeyPointProcess(kpFile))
  kptProcess.append(keyPoint)
  #Extracting the images
  imgArray=imageProcessing(imgFile)
  imgProcess.append(imgArray)

#print(f'KEYPOINT SHAPE:{np.asarray(kptProcess).shape}')
#print(f'IMAGES SHAPE:{np.array(imgProcess).shape}')

In [ ]:
plt.figure(figsize=(30,30))
i,idx=1,1
for num in range(0,16):
  #idx=np.random.choice(kpA.shape[0])
  plt.subplot(4,4,i)
  plt.imshow(imgA[idx])
  x,y=kpA[idx][:,0], kpA[idx][:,1]
  plt.scatter(x, y, color='red', s=10)
  plt.title(idx)
  i+=1
  idx+=1

## **PICKLING THE TEST IMAGES AND TEST LABELS**

In [ ]:
'''featurefile=open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/IMG/TEST_Fimg128_denoise_hist.pickle', 'wb')
pickle.dump(imgProcess,featurefile)
featurefile.close()
'''
IMGfile=pickle.load(open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/IMG/TEST_Fimg128_denoise_hist.pickle','rb'))
imgA=np.asarray(IMGfile)
print(imgA.shape)

(128, 128, 64)


In [ ]:
'''featurefile=open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/KEYPOINT/TEST_Fkp128_denoise_hist.pickle', 'wb')
pickle.dump(kptProcess, featurefile)
featurefile.close()
'''
KPfile=pickle.load(open('/content/drive/MyDrive/Colab Notebooks/scoliosis xray Single View/boostnet_labeldata/SmallAugmentFiles/Feature/KEYPOINT/TEST_Fkp128_denoise_hist.pickle','rb'))
kpA=np.asarray(KPfile)
print(kpA.shape)

(128, 68, 2)
